In [ ]:
from ngsolve import *

from ngsolve.meshes import *
from ngsolve.solvers import *
from ngsolve.webgui import Draw

import numpy as np
import matplotlib.pyplot as plt

#from su2_yangmills import *
from lgt import *

In [ ]:
ne=4
#mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=True, periodic_y=True)
mesh = MakeStructured2DMesh(quads=True, nx=ne, ny=ne, periodic_x=False, periodic_y=False)
Draw(mesh)

In [ ]:
order=1
fesC = H1(mesh, order=order)**3 # for su2 charge
fesJ = HCurl(mesh, order=order-1)**3 # for su2 gauge potential

# discrete connection one form sitting on borders
fesA = FacetFESpace(mesh, order=0)**3

# parallel transport maps between neighbouring volumes, belonging to an oriented interface
#TODO: orientation?
fesU = FacetFESpace(mesh, order=0)**4

# space of wilson loops centered around bones (= corners (2D) or edges (3D))
fesW = H1(mesh, order=0)**4

# space of action integrand
fesS = H1(mesh, order=0)

# discrete curvature sitting at bones (= corners (2D) or edges (3D))
fesB = H1(mesh, order=0)**3

In [ ]:
fesT = H1(mesh, order=0)
gfT = GridFunction(fesT)

e_id = 0
edge = mesh.edges[e_id]
v_of_e = edge.vertices
for v in v_of_e:
    print(v)
    gfT.vec[v.nr] = 1

Draw(gfT, mesh)

In [ ]:
gfv = GridFunction(fesT)
gfvedges = GridFunction(fesT)

v_id = 0
v = mesh.vertices[v_id]
vedges = get_oriented_vedges_tan(v, mesh)
vedges = [edge_or_angle[0] for edge_or_angle in vedges]
print(vedges)
print(get_vedges(v, mesh))

gfv.vec[v_id] = 1.
for e_id in vedges:
    edge = mesh.edges[e_id]
    
    for v in edge.vertices:
        gfvedges.vec[v.nr] = 1
        
    Draw(gfvedges, mesh)
    gfvedges.vec[:] = 0

Draw(gfv, mesh)
#Draw(gfvedges, mesh)

In [ ]:
gfU = GridFunction(fesU)
#gfU.components[0].vec[:] = 1

phi  = 0.1*pi
gfU.components[0].vec[:] = cos(phi)
gfU.components[3].vec[:] = sin(phi)

#phi = 0.3*pi
#for e in range(mesh.nedge):
#    phi_e = float(e)/float(mesh.nedge)*phi
#    gfU.components[0].vec[e] = cos(phi_e)
#    gfU.components[3].vec[e] = sin(phi_e)


In [ ]:
v_id = 4
e_id = 7
v = mesh.vertices[v_id]
e_start = mesh.edges[e_id]

print(v)
print(v.edges)
print(e_start)
#if e_start not in v.edges:
#    raise ValueError(f"edge {estart} not at vertex {v}")

#get_wilson_loop(gfU, v, mesh, start_edge=e_start)
qWv = get_wilson_loop(gfU, v, mesh, start_edge=None)
print(np.arccos(qWv[0]))

In [ ]:
gfW = GridFunction(fesW)
#for v in mesh.vertices:
#    gfU.components[0].vec[v.nr] = v.nr+1

calc_gfW(gfU, gfW, mesh)

In [ ]:
#Draw(gfW.components[0], mesh)
# values inbetween are interpolated and have no meaning!
# norm inbetween is not 1 due to linear interpolation
Draw(Norm(gfW), mesh)

In [ ]:
gfS = GridFunction(fesS)
#calc_lnwilson_action(gfW, gfS)
calc_wilson_action(gfW, gfS)

In [ ]:
Draw(gfS, mesh)

In [ ]:
gfW_ah = tr_ah(su2mat_from_q(gfW))
Draw(gfW_ah[2], mesh)
Draw((-vec_from_q(gfW)- gfW_ah)[2], mesh)